In [1]:
import os
import glob
import time
import numpy as np
import pyprismatic as pr

In [2]:
basename = glob.glob('W_supercell*.xyz')[0]

In [3]:
sliceThickness = 1.58
tileX = tileY = tileZ = 1
E0 = 100 # in keV
realspacePixelSizeX = realspacePixelSizeY = 0.05
phonons = 10
algorithm = 'prism'

probeDefocus_list = np.arange(-60, 60, 20)
probeDefocus_list = [0]
probeStepX = probeStepY = 0.2  # in A
probeSemiangle = 35  # in mrad
alpha_factor = 1.1
C3 = 1.39E5  # in angstrom
C5 = -1.31E8  # in angstrom
scanWindowXMin = scanWindowYMin = 0.0  # fractional coordinates
scanWindowXMax = scanWindowYMax = 0.9999  # fractional coordinates
interpolationFactor = 2

In [ ]:
for probeDefocus in probeDefocus_list:
    filenameAtoms = f"{basename}"
    base_name = f"{basename}-def{probeDefocus}-TDS{phonons}"
    interpolationFactorX = interpolationFactorY = interpolationFactor
    alphaBeamMax = probeSemiangle * alpha_factor
    if algorithm == 'prism':
        base_name = f"{base_name}-{algorithm}{interpolationFactor}"
    includeThermalEffects = False if phonons == 0 else True
    filenameOutput = f"{base_name}.emd"

    meta = pr.Metadata(
        interpolationFactorX=interpolationFactorX,
        interpolationFactorY=interpolationFactorY,
        filenameAtoms=filenameAtoms,
        filenameOutput=filenameOutput,
        realspacePixelSizeX=realspacePixelSizeX,
        realspacePixelSizeY=realspacePixelSizeY,
        sliceThickness=sliceThickness,
        potBound=2,
        numFP=phonons if includeThermalEffects else 1,
        tileX=tileX,
        tileY=tileY,
        tileZ=tileZ,
        E0=E0,
        alphaBeamMax=alphaBeamMax,
        probeStepX=probeStepX,
        probeStepY=probeStepY,
        probeDefocus=probeDefocus,
        C3=C3,
        C5=C5,
        probeSemiangle=probeSemiangle,
        detectorAngleStep=1,
        probeXtilt=0,
        probeYtilt=0,
        scanWindowXMin=scanWindowXMin,
        scanWindowYMin=scanWindowYMin,
        scanWindowXMax=scanWindowXMax,
        scanWindowYMax=scanWindowYMax,
        randomSeed=13550,
        algorithm=algorithm,
        includeThermalEffects=includeThermalEffects,
        alsoDoCPUWork=True,
        save2DOutput=False,
        save3DOutput=True,
        save4DOutput=False,
        saveDPC_CoM=False,
        savePotentialSlices=False,
        integrationAngleMin=60,
        integrationAngleMax=120,
        numThreads=30,
        batchSizeTargetCPU=1,
        batchSizeTargetGPU=2,
        numStreamsPerGPU=3,
    )
    meta.writeParameters(f"{base_name}.txt")
    start = time.time()
    meta.go()
    end = time.time()
    formatted_time = time.strftime("%H:%M:%S", time.gmtime(end-start))
    total_time = f"The simulation time of {filenameOutput} was: {formatted_time} ({end-start:6g} s)"
    print(total_time)
    with open(f"{base_name}-timing.txt", 'w') as f:
        f.write(total_time)